<a href="https://colab.research.google.com/github/jdmedinatobon/proyectoMachineLearning/blob/master/CrearArchivoDescriptores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mahotas

In [1]:
import os
import numpy as np
import nibabel as nib
from zipfile import ZipFile
import csv
import mahotas as mh
import tensorflow as tf
import time

In [6]:
#Se descomprime el archivo con los datos preprocesados
!unzip PruebaDatos.zip #En verdad no es este archivo.
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Archive:  PruebaDatos.zip
   creating: PruebaDatos/
   creating: PruebaDatos/func/
 extracting: PruebaDatos/func/sub-01_task-LTM_run-1_bold.nii.gz  
  inflating: PruebaDatos/func/sub-01_task-LTM_run-1_events.tsv  
Found GPU at: /device:GPU:0


In [0]:
pathArchivoDatosPreprocesados = "PruebaDatos" #Ruta del archivo raiz de los datos preprocesados

sub = [0, 0]
run = 1

def darNombreArchivoImagenes(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_bold.nii.gz"

def darNombreArchivoEtiquetas(pSub, pRun):
  return "PruebaDatos/func/sub-" + str(pSub[0]) + str(pSub[1]) + "_task-LTM_run-" + str(pRun) + "_events.tsv"

def obtenerImagenes(pImagenes):
  
  imagenes = np.zeros((64,64,30,30))

  for indice in range(0,15):
    for i in range(0,64):
      for j in range(0,64):
          for k in range(0,30):
              imagenes[i][j][k][2*indice] = pImagenes[i][j][k][1+indice*13]
              imagenes[i][j][k][2*indice+1] = pImagenes[i][j][k][2+indice*13]

  return imagenes

#object -> 0
#place -> 1
#face -> 2
def leerEtiquetas(pArchivoEtiquetas):
  etiquetas = []

  with open(pArchivoEtiquetas) as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    for row in reader:
      
      etiqueta = row['trial_type']

      if(etiqueta == 'object'):

        etiquetas.append(0)
        etiquetas.append(0)
      elif(etiqueta == 'place'):

        etiquetas.append(1)
        etiquetas.append(1)
      elif(etiqueta == 'face'):

        etiquetas.append(2)
        etiquetas.append(2)
      else:
        print("Error en el formato.")


  return etiquetas

def importarDatos():
  datosPreprocesados = np.zeros((64,64,30,1800))
  etiquetasPreprocesados = np.zeros(1800)

  contador = 0
  for s1 in range(1,2):#(1,11)

    if(s1 == 10):
      sub[0] = 1
      sub[1] = 0
    else:
      sub[1] = s1

      for r in range(1,2):#(1,7)
        run = r
        archivoImagenes = darNombreArchivoImagenes(sub, run)
        archivoEtiquetas = darNombreArchivoEtiquetas(sub, run)

        imagenes = nib.load(archivoImagenes).get_fdata()
        etiquetas = leerEtiquetas(archivoEtiquetas)

        muestras = obtenerImagenes(imagenes)

        for indice in range(0,30):
          for i in range(0,64):
            for j in range(0,64):
                for k in range(0,30):
                  datosPreprocesados[i][j][k][30*contador+indice] = muestras[i][j][k][indice]
                  etiquetasPreprocesados[30*contador+indice] = etiquetas[indice]

        contador+=1

    return datosPreprocesados, etiquetasPreprocesados

In [3]:
print("Importando datos...")
test, testet=importarDatos()

Importando datos...


In [0]:
escala = 256

test2 = test[:,:,:,1]
test2 = (test2/test2.max())*escala
test2 = test2.astype(int)
#test2 = test2-1

start = time.time()
print("Calculando descriptores...")
descriptor = mh.features.haralick(test2, ignore_zeros=True, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=True, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)

print(descriptor)
print("Tiempo: {}".format(time.time()-start))

Calculando descriptores...


In [6]:
Calculando descriptores...
[ 3.85918826e-02  4.90384291e+01  6.13790632e-01  6.38849266e+01
  4.35716991e-01  8.70680740e+00  2.06501277e+02  4.08582836e+00
  6.04907255e+00  7.63875133e-04  2.85176985e+00 -1.23717111e-01
  7.39392946e-01]
Tiempo: 0.08932089805603027

array([[116, 129,  89],
       [ 53, 135, 114],
       [ 42, 105, 134]])

In [12]:
test.shape

(64, 64, 30, 1800)

In [7]:
2**12

4096

In [7]:
test2 = (test2/test2.max())*256

17127

In [16]:
print(test2[:,:,12])

[[  111    57   212   171   188    59   110   177   234    75    34   116
     60   284   204   189   262   189   176   161]
 [   55   121   127    18   143    93    27   147   121    44   278    96
    204   257   164   197   126   241   118   343]
 [  172   132   118    99   102    92    19   172   185   247    88    59
    120   254   158   134   207   291   149   189]
 [  163    80   106    74   226   314   192    92   189   152   118    85
      2   242   186   156    61    78    63   292]
 [  164    67   169   206   150    75    68   116   230   133    16   188
    204   342   123   182   292   173    35   208]
 [  178   219    86   127   160    51   180    62    70   180   143    87
     42   228   297   241    71   130   162   270]
 [  104    34   135    78   173   126   184   113    32   122    40   113
     81   283   120   151   165   214   128   216]
 [  215   150   172    91   116   159    52   206   249   315   118    88
    131   218    31   191   120   469   371   311]


In [13]:
65000*65000

4225000000

In [17]:
4096*4096*4

67108864

In [19]:
4225000000/(16777216*4)

62.95740604400635

In [20]:
1800/60

30.0

In [22]:
8192*2

16384

In [0]:
test2 = (test2/test2.max())*8192

In [7]:
test2 = test[:,:,:,:]
test2.max()

25586.0

In [5]:
2**16

65536

In [6]:
2**14

16384